In [ ]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("stocksTicksData")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/04/01 01:09:06 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/04/01 01:09:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bf17613e-6b28-49e5-85b7-4f4285d359da;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 958ms :: artifacts dl 18ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [3]:

StockTicksDf = spark.read.format("csv")\
                .option("header", True)\
                .option("inferschema",True)\
                .load("hdfs://localhost:9000/layers/raw/stocks-ticks-csv")

StockTicksDf.printSchema()
StockTicksDf.show(2)


root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- _symbol: string (nullable = true)

+--------+-----+------+-------------------+----+-----+---+----+--------+
|  symbol|price|volume|          timestamp|year|month|day|hour| _symbol|
+--------+-----+------+-------------------+----+-----+---+----+--------+
|HDFCBANK| 38.0|    10|2022-03-27 13:44:00|2022|    3| 27|  13|HDFCBANK|
|HDFCBANK| 40.0|    40|2022-03-27 13:44:00|2022|    3| 27|  13|HDFCBANK|
+--------+-----+------+-------------------+----+-----+---+----+--------+
only showing top 2 rows



In [4]:
# Kafka value, key are in binary format
# we need to type cast to STRING, 
# we pick only value, timestamp as we need for 10 minute window
# ticksDf = StockTicksDf.selectExpr("CAST(value AS STRING)", "timestamp")
# ticksDf.printSchema() # we get only value as string

In [5]:
# import pyspark.sql.functions as F
# from pyspark.sql.types import StructField, StructType, DoubleType, StringType, LongType, TimestampType

# schema = StructType( [
#     StructField("symbol", StringType(), True),
#     StructField("price", DoubleType(), True),
#     StructField("volume", LongType(), True),
#     StructField("timestamp", LongType(),  True)
# ])

In [6]:
# we parse value which is JSON string to kafka struct, 
# this create a new column [replacing existing value which is string] with type
#jsonDf = StockTicksDf.withColumn("value", F.from_json("value", schema))
#jsonDf.printSchema()

In [7]:
# now extract all the sub fields symbol, price, volume, timestamp into data frame
# exisitng value, timestamp shall be removed. existing timestamp is value produced time, not a stock tick time
# value.timestamp that comes from producer is basically a event time
#  

In [14]:
StockTicksDf.write.mode("overwrite")\
                .parquet("hdfs://localhost:9000/layers/raw/stocks-ticks-parquet")

In [16]:
StockTicksDf11 = spark.read.format("parquet")\
                .load("hdfs://localhost:9000/layers/raw/stocks-ticks-parquet")
StockTicksDf11.printSchema()
StockTicksDf11.show()

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- _symbol: string (nullable = true)

+----------+-----+------+-------------------+----+-----+---+----+----------+
|    symbol|price|volume|          timestamp|year|month|day|hour|   _symbol|
+----------+-----+------+-------------------+----+-----+---+----+----------+
|OBEROIRLTY| 41.0|    30|2022-03-27 22:24:00|2022|    3| 27|  22|OBEROIRLTY|
|OBEROIRLTY| 18.0|    40|2022-03-27 22:25:00|2022|    3| 27|  22|OBEROIRLTY|
|IDFCFIRSTB| 36.0|    10|2022-03-27 19:57:00|2022|    3| 27|  19|IDFCFIRSTB|
|IDFCFIRSTB| 50.0|    40|2022-03-27 19:58:00|2022|    3| 27|  19|IDFCFIRSTB|
|MCDOWELL-N| 37.0|    10|2022-03-27 23:42:00|2022|    3| 27|  23|MCDOWELL-N|
|MCDOWELL-N| 41.0|    10|2022-03-

In [10]:
# we have data coming in stream, each record has timestamp
# we have milli seconds 1647283268253, the actual seconds Mon Mar 14 2022 14:41:08
# now we have to calculate 1 minute window, sum(volume) up to 1 minute
# after conversions, we take timestamp which is truncatd to minute instead of milli seconds, drop temp columns
# stockTickDf1  = StockTicksDf \
#                 .withColumn("timestampTemp", (F.col("timestamp") / 1000).cast("timestamp"))\
#                 .withColumn("trade_time", F.date_trunc("minute", F.col("timestampTemp")))\
#                 .drop("timestamp")\
#                 .drop("timestampTemp")\
#                 .withColumnRenamed("trade_time", "timestamp")
              

In [11]:
# from pyspark.sql.functions import date_format

# stockTickDf = stockTickDf .withColumn("Year", date_format("TIMESTAMP", "yyyy"))\
#                      .withColumn("Month", date_format("TIMESTAMP", "MM"))\
#                      .withColumn("Day", date_format("TIMESTAMP", "dd"))\
                     #.withColumn("Hours",date_format("TIMESTAMP","hh"))\
                     #.withColumn("Minutes",date_format("TIMESTAMP","mm"))


In [12]:
#stockTickDf .write.mode("overwrite")\
#            .partitionBy("Year", "Month", "Day","Hours","Minutes","symbol")\
 #           .format("csv")\
  #          .save("hdfs://localhost:9000/stocks-ticks")
            
#stockTickDf .printSchema()
#stockTickDf .show(5)

In [20]:
# aggregate
# sum (volume), sum(traded_value), candles open, high, low, close [OHLC]
# groupBy by symbol, timestamp

# now add delayed record processing with waterMark
# stockTickDf_1 = StockTicksDf11\
#                             .selectExpr("timestamp", "symbol",  "struct(*) as value")\
#                             .groupBy("symbol", window(col("timestamp", "60 seconds")))\
#                             .agg( collect_list(col("value").alias("ticks")))

# stockTickDf_1.printSchema()

NameError: name 'window' is not defined

In [ ]:
def processBatchData(candleBatchDf, batch_id):
    print ("process batch called", batch_id, "writing ", candleBatchDf.count())
    
    (
     candleBatchDf
        .partitionBy("year", "month", "day", "hour", "SYMBOL")
        .write
        .mode("append")
        .format("parquet")
        .filter("price">0, "volume">0)
        .save("hdfs://localhost:9000/layers/clean/stocks-ticks-parquet")
    )

In [ ]:
# spark stream with batch output
# spark basically process the data as batch
# for developer convinence, we have api, that can give us finite data frame for writing to file/jsbc/db/custom coding
# process discrete output batch
# this function is called for every update/delete/comlete triggers with result data set as dataframe
# candleBatchDf is BATCH DATAFRAME
# stockTickDf1Min is STREAM DATAFRAME


    # result = candleBatchDf.collect()
    # print(result)
    # candleBatchDf.show(2)
    # write to csv/json/parqeut/database /jdbc etc
    # window is nested column, has its children columns called start and end
    # start and end are part of nested window column
    # append mode will not delete existin data, instead append to existing table

# stockTickDf_2 = ( StockTicksDf11.write.mode("overwrite")
#                                  .withColumn("year", F.date_format(F.col("timestamp"), "yyyy"))
#                                  .withColumn("month", F.date_format(F.col("timestamp"), "MM"))
#                                  .withColumn("day", F.date_format(F.col("timestamp"), "dd"))  
#                                  .withColumn("hour", F.date_format(F.col("timestamp"), "HH"))   
#                                  .withColumn("_symbol", F.col("symbol"))   
#                                  .partitionBy("year", "month", "day", "hour", "_symbol")
#                                  .outputMode("append")
#                                  .format("parquet")
#                                  .option("header", True)
#                                  .filter(col("price")>0 & col("volume")>0)
#                                  .save( "hdfs://localhost:9000/layers/clean/stocks-ticks-parquet")
#                     )
 


       

    
